In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [1]:
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os
from tqdm import tqdm

## Read Data

In [ ]:
# Run this cell if you are using Google Colaboratory

# from google.colab import drive
# drive.mount('/content/drive')

# BASE_DIR = Path(f"/content/drive/MyDrive/{your_path}")
# DATA_DIR = BASE_DIR.joinpath("data")  # Please upload the data_quality_estim.csv dataset to the data folder 

# FILE_NAME = "data_quality_estim.csv"

# FILE = DATA_DIR.joinpath(FILE_NAME)

In [6]:
BASE_DIR = Path('Coherence_of_Words_2classes_Mistral-7B-Instruct-v.0.1_4_bit.ipynb').resolve().parent.parent

DATA_DIR_NAME = 'data/datasets'  # Local directory

DATA_DIR = BASE_DIR.joinpath(DATA_DIR_NAME)
FILE_NAME_QUAL = 'data_quality_estim.csv'

FILE = DATA_DIR.joinpath(FILE_NAME_QUAL)

In [7]:
data = pd.read_csv(FILE)
data.head(10)

,task,agg_label
0,able amaze fill glad stop daily fantastic move...,bad
1,able amaze fill stop glad daily fantastic move...,bad
2,able amaze fill stop glad daily fantastic move...,bad
3,able amaze glad fill stop fantastic daily move...,bad
4,absolute church truth doug scripture symbol li...,good
5,absolute church truth mission nasa doug mars e...,bad
6,absolute mission church nasa mars earth truth ...,good
7,absolutely fast white rice service noodle terr...,bad
8,absolutely fast white super service customer r...,good
9,absolutely grain experience soon there's proce...,bad


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   task       3861 non-null   object
 1   agg_label  3861 non-null   object
dtypes: object(2)
memory usage: 60.5+ KB


In [ ]:
!nvidia-smi

Sun Mar 24 19:47:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Check if cuda is available
assert torch.cuda.is_available(), "Cuda is not available, you cannot use the quantized version of the Mistral model without a GPU!"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Pipeline of the Inference

In [ ]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [ ]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [ ]:
def few_shot_pipeline(
        instruction, 
        sample, 
        model, 
        device='cpu', 
        temp=0.1, 
        n_token=100, 
        print_system_prompt=False, 
        save_system_prompt=False
        ):
    
    # Construct prompt
    prompt = f"""<s> [INST] {instruction}\n{sample} // [/INST]"""
    if print_system_prompt:
        print(prompt)
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    if save_system_prompt:
        result = answer[0]
    else:
        result = extract_substring(answer[0])
    return result.replace("\n", "")


## Prompt Engineering

#### Type I. 3-shot

In [ ]:
prompt = """<s> [INST] bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // good
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // bad
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // good
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])


<s>  [INST] bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // good
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // bad
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // good
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] It appears that you have provided a list of words that may be related to various topics such as firearms, knives, and criminal activity. However, it is important to note that the context in which these words are used can greatly impact their meaning and moral implications.

For example, the word "weapon" can be used to refer to a tool used for self-defense or to harm others, depending on the context. Similarly, the word "murder" can be


**Conclusion:** The model does not quite understand what task needs to be completed

#### Type II. 3-shot

In [ ]:
prompt = """<s> [INST] bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // strong connection

appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // weak connection

bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // strong connection

april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=500)

print(answer[0])

<s>  [INST] bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // strong connection

appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // weak connection

bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // strong connection

april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] It seems like you are trying to connect different topics and ideas in your sentences. Here are some possible ways to rephrase your sentences to make them more coherent:

* Bike, firearm, knife, criminal, carry, death, weapon, scsi, murder, smith, handgun, moral, child, defense, tape, strong connection.
	+ Bike, firearm, knife, and criminal carry are often associated with death and murder. Smith handguns are a popular choice for self-defense, but there are moral considerations to take into account when ca

**Conclusion:** The model tries to understand what is wanted from her, instead of the required answer, she tries to find a connection between words and describe it

#### Type III. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] You will receive a list of words, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the presented word set or at least for the most part of the set?"
bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the presented word set or at least for the most part of the set?"
bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] YES</s>


**Conclusion:** In this example, the Chain-of-Tough technique was used – we explained to the model how she needs to think / answer the question. The model realized that it was required, and this prompt can be used for further research

#### Type IV. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] You will receive a list of words, and you should answer "YES" if words have a strong connection between them or “NO” in the opposite case.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words, and you should answer "YES" if words have a strong connection between them or “NO” in the opposite case.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape - YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx - NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith - YES
april nasa star canada bike score always goal maynard area player hockey wings cente

**Conclusion:** Unstable prompt, the model roughly understands the form of the response, but gives answers for all examples

#### Type V. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=500)

print(answer[0])

<s>  [INST] Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] YES. It is possible to determine a common topic for the presented word set. The most part of the set is related to the topic of sports, specifically hockey. The words "bi

**Conclusion:** The model gives an answer for all words, and also tries to determine the topic to which these words relate

#### Type V_v2. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

**Conclusion:** In comparison with the previous prompt, a clarification has been added about the form of the response. Prompt is suitable for further analysis

#### Type VI. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] You are a human. Select one of four categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

<s>  [INST] You are a human. Select one of four categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] YES. The common topic for the presented word set is "vehicles" and "sports."</s>


**Conclusion:** In this prompt, refinement was used to try to emulate human behavior by clarifying the role for LLM. As you can see, the model gives a detailed response.

#### Type VI_v2. 3-shot Chain-of-Thought

In [ ]:
prompt = """<s> [INST] You are a human. Select one of four categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

<s>  [INST] You are a human. Select one of four categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES
april nasa star canada bike score always goal maynard area player hockey wings center white // [/INST] YES</s>


**Conclusion:** A clarification about the form of the response was added to the previous version of the prompt. Prompt can be used for further research.

## Inference on the Entire Dataset

In [ ]:
test_data = data.sample(frac=1, random_state=42)

### Type III. 3-shot Chain-of-Thought

In [ ]:
INSTRUCTION = """You will receive a list of words, and you should answer "YES" or "NO" to the question: "Is it possible to determine a common topic for the presented word set or at least for the most part of the set?"
bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES"""

SAMPLE = data.iloc[0]["task"]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda")

res

' NO</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_mistral_7b_4bit_type3.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)

for i in tqdm(range(len(test_data))):
    SAMPLE = test_data.iloc[i]["task"]
    res_true = test_data.iloc[i]["agg_label"]
    res_model = few_shot_pipeline(
        INSTRUCTION,
        SAMPLE,
        model,
        device="cuda",
        temp=0.1,
        n_token=200
    )
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


100%|██████████| 3861/3861 [3:12:10<00:00,  2.99s/it]


### Type V_v2. 3-shot Chain-of-Thought

In [ ]:
INSTRUCTION = """Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES"""

SAMPLE = data.iloc[80]["task"]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", n_token=100)

res

' YES</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_mistral_7b_4bit_type5.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)

with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(test_data))):
        SAMPLE = test_data.iloc[i]["task"]
        res_true = test_data.iloc[i]["agg_label"]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.1,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


100%|██████████| 3861/3861 [2:04:51<00:00,  1.94s/it]


### Type VI_v2. 3-shot Chain-of-Thought

In [ ]:
INSTRUCTION = """You are a human. Select one of two categories-characteristics of the provided set of tokens by answering the question ”Is it possible to determine a common topic for the presented word set or at least for the most part of the set?”.
You could answer one of the following: YES - if you agree with the statement and words have a strong connection between them, NO - when words seem to be unconnected. Please, make sure you to only return YES or NO and nothing more.

bike firearm knife criminal carry death weapon scsi murder smith handgun moral child defense tape // YES
appreciate advance anybody info thank experience hello greatly suggestion wonder edge able email recently thanx // NO
bible christ matthew heaven scripture lord father word passage teach mary spirit doctrine verse faith // YES"""

SAMPLE = data.iloc[0]["task"]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", n_token=100)

res

' YES</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_qality_estim_mistral_7b_4bit_type6.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)

with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(test_data))):
        SAMPLE = test_data.iloc[i]["task"]
        res_true = test_data.iloc[i]["agg_label"]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.1,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_true};{res_model}\n")


 94%|█████████▍| 3635/3861 [1:58:47<07:31,  2.00s/it]